In [1]:
!pip install ortools==9.7.2996 pandas==2.1.0

In [120]:
from ortools.sat.python import cp_model

In [121]:
def resolve(solucionador, modelo, estacionamentos, avioes):
  status = solucionador.Solve(modelo)
  print(solucionador.StatusName(status))
  if status == cp_model.INFEASIBLE:
    print("Sem solucao")
    return
  for estacionamento in estacionamentos:
    variavel = estacionamento.variavel
    valor = solucionador.Value(variavel)
    if valor == 0:
      print(f"{variavel} sem aviao")
    else:
      aviao = avioes[valor - 1]
      print(f"{variavel} tem aviao {valor} grande={aviao.grande}")


In [122]:
def avioes_distintos(estacionamentos, modelo):
  variaveis = [estacionamento.variavel for estacionamento in estacionamentos]
  modelo.AddAllDifferent(variaveis)

In [123]:
def todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo):
  variaveis = {}
  for i in range(1, total_de_avioes + 1):
    for j, estacionamento in enumerate(estacionamentos):
      aviao_i_em_j = modelo.NewBoolVar(f'aviao_{i}_em{j}')
      modelo.Add(estacionamento.variavel == i).OnlyEnforceIf(aviao_i_em_j)
      modelo.Add(estacionamento.variavel != i).OnlyEnforceIf(aviao_i_em_j.Not())
      variaveis[(i, j)] = aviao_i_em_j

  for i in range(1, total_de_avioes + 1):
    modelo.AddExactlyOne([variaveis[(i,j)] for j in range(len(estacionamentos))])


In [124]:
class Estacionamento:
  def __init__(self, k, total_de_avioes, grande, modelo):
    self.grande = grande
    self.variavel = modelo.NewIntVar(0, total_de_avioes, f'estacionamento_{k}')
    self.k = k
    self.vizinhos = []


In [125]:
class Aviao:
  def __init__(self, k, grande):
    self.k = k
    self.grande = grande

In [126]:
def remover_do_estacionamento(modelo, variavel_estacionamento, avioes_grandes):
  for aviao in avioes_grandes:
    modelo.Add(variavel_estacionamento != aviao.k)

In [127]:
def limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes):
  avioes_grandes = [aviao for aviao in avioes if aviao.grande]
  for estacionamento in estacionamentos:
    if not estacionamento.grande:
      # nao aceita!!
      # REMOVER AVIOES GRANDES!
      remover_do_estacionamento(modelo, estacionamento.variavel, avioes_grandes)

In [130]:
avioes = [Aviao(1, True),
          Aviao(2, False),
          Aviao(3, True)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo),
                   Estacionamento(2, total_de_avioes, False, modelo),
                   Estacionamento(3, total_de_avioes, True, modelo),
                   Estacionamento(4, total_de_avioes, True, modelo)]
estacionamentos[2].vizinhos = [estacionamentos[3]]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes)

OPTIMAL
estacionamento_1 tem aviao 2 grande=False
estacionamento_2 sem aviao
estacionamento_3 tem aviao 3 grande=True
estacionamento_4 tem aviao 1 grande=True


In [ ]:
def limita_vizinhos(modelo, estacionamentos, avioes):
  pass